In [2]:
pip install pyscipopt

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install geopy

Note: you may need to restart the kernel to use updated packages.


In [5]:
from pyscipopt import Model, quicksum, multidict
import numpy as np
import pandas as pd
import geopandas as gpd
import random
import json
from geopy.distance import geodesic

In [5]:
# adapted from https://scipbook.readthedocs.io/en/latest/flp.html
def flp(I,J,d,M,c,existing_sites=None):
    model = Model("flp")
    x,y = {},{}
    for j in J:
        y[j] = model.addVar(vtype="B", name="y(%s)"%j)
        for i in I:
            x[i,j] = model.addVar(vtype="C", name="x(%s,%s)"%(i,j))
    for i in I:
        model.addCons(quicksum(x[i,j] for j in J) == d[i], "Demand(%s)"%i)
    for j in M:
        model.addCons(quicksum(x[i,j] for i in I) <= M[j]*y[j], "Capacity(%s)"%i)
        model.addCons(quicksum(x[i,j] for i in I) >= 0.7 * M[j] * y[j], "MinCapacityUse(%s)"%j) # ensures no school has capacity under 70%
    for (i,j) in x:
        model.addCons(x[i,j] <= d[i]*y[j], "Strong(%s,%s)"%(i,j))
    
    if existing_sites:
        for j in existing_sites:
            model.addCons(y[j] == 1, name=f"ForceOpen({j})")

    model.addCons(quicksum(y[j] for j in J) <= 6, "FacilityLimit") 
            
    model.setObjective(
        quicksum(c[i,j]*x[i,j] for i in I for j in J),
        "minimize")
    model.data = x,y
    return model

In [7]:
# for I, d make a dictionary of planning units to number of students
pu = gpd.read_file('/Users/leahwallihan/Durham_school_planning/DPS-Planning/GIS_files/hs_full_geo.geojson').set_index('pu_2324_84')
pu = pu.to_crs('EPSG:4326')

In [7]:
pu_data = pu['basez'].to_dict()
I, d = multidict(pu_data)

In [9]:
# for J, M make a dictionary of sites to capacities
schools = gpd.read_file('/Users/leahwallihan/Durham_school_planning/DPS-Planning/GIS_files/dps_hs_locations.geojson')
schools = schools.to_crs('EPSG:4326')

# find which planning units have existing school
schools['pu'] = None

for i, geometry in enumerate(pu['geometry']):
    in_geometry = geometry.contains(schools['geometry'])
    pu_id = pu.index[i]

    schools.loc[in_geometry, 'pu'] = pu_id

In [9]:
# let's remove planning units in downtown from J to make problem simpler
not_central = pu[(pu['Region'] != 'Central')]

# initialize dictionary of planning units with capacity of 1600 for potential site
pu_dict = {}
for idx, row in not_central.iterrows():
    pu_dict[idx] = 1850

# replace capacities of planning units with existing schools
pu_dict[45] = 1600
pu_dict[507] = 1810
pu_dict[602] = 1240 # reduce by 300 for choice?
pu_dict[566] = 1540
pu_dict[290] = 1235 # reduce by 300 for choice?

J, M = multidict(pu_dict)

# define which sites already exist
existing_sites = {602, 290, 45, 566, 507}

In [10]:
# Get centroids and convert to lat/lon tuples
centroid_coords = {
    idx: (geom.y, geom.x)  # (latitude, longitude)
    for idx, geom in pu.geometry.centroid.items()
}

# Now build the distance matrix using geodesic distances
c = {}
for i in I:
    for j in J:
        c[i, j] = geodesic(centroid_coords[i], centroid_coords[j]).miles

/var/folders/ml/wj07z83j0yq4rjs8n_7sv0mw0000gn/T/ipykernel_61541/482943662.py:4: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  for idx, geom in pu.geometry.centroid.items()


In [11]:
'''
# for testing:
I_small = random.sample(I, 100)
d_small = {i: d[i] for i in I_small}
c_small = {(i,j): c[i,j] for i in I_small for j in J if (i,j) in c}

model = flp(I_small, J, d_small, M, c_small, existing_sites=existing_sites)
x,y = model.data
model.setParam('limits/solutions', 3)
model.optimize()
EPS = 1.e-6
edges = [(i,j) for (i,j) in x if model.getVal(x[i,j]) > EPS]
facilities = [j for j in y if model.getVal(y[j]) > EPS]
print ("Optimal value=", model.getObjVal())
print ("Facilities at nodes:", facilities)
print ("Edges:", edges)
'''

'\n# for testing:\nI_small = random.sample(I, 100)\nd_small = {i: d[i] for i in I_small}\nc_small = {(i,j): c[i,j] for i in I_small for j in J if (i,j) in c}\n\nmodel = flp(I_small, J, d_small, M, c_small, existing_sites=existing_sites)\nx,y = model.data\nmodel.setParam(\'limits/solutions\', 3)\nmodel.optimize()\nEPS = 1.e-6\nedges = [(i,j) for (i,j) in x if model.getVal(x[i,j]) > EPS]\nfacilities = [j for j in y if model.getVal(y[j]) > EPS]\nprint ("Optimal value=", model.getObjVal())\nprint ("Facilities at nodes:", facilities)\nprint ("Edges:", edges)\n'

In [ ]:
model = flp(I, J, d, M, c, existing_sites=existing_sites)
model.setParam('limits/solutions', 5)
model.optimize()
EPS = 1.e-6
x,y = model.data
edges = [(i,j) for (i,j) in x if model.getVal(x[i,j]) > EPS]
facilities = [j for j in y if model.getVal(y[j]) > EPS]
print ("Optimal value=", model.getObjVal())
print ("Facilities at nodes:", facilities)
print ("Edges:", edges)

In [ ]:
solution_reports = []

# Get all stored solutions
sols = model.getSols()

for sidx, sol in enumerate(sols):
    assignments = {}

    for (i_, j_) in x:
        if model.getSolVal(sol, x[i_, j_]) > 0.5:
            if j_ not in assignments:
                assignments[j_] = []
            assignments[j_].append(i_)

    student_count = {}
    if 'students' in globals():  
        for j_, pus in assignments.items():
            student_count[j_] = sum(students.get(i_, 0) for i_ in pus)

    solution_reports.append({
        'solution_number': sidx + 1,
        'facilities': list(assignments.keys()),
        'assignments': assignments,
        'student_count': student_count if 'students' in globals() else None
    })

In [118]:
for report in solution_reports:
    print(f"\n--- Solution #{report['solution_number']} ---")
    print("Facilities opened:", report['facilities'])

    print("Assignments:")
    for fac, pus in report['assignments'].items():
        print(f"  Facility {fac} <-- Planning Units {pus}")

    print("Student Count per Facility:")
    student_counts = {}
    for fac in report['facilities']:
        count = 0
        for pu_id in report['assignments'][fac]:
            count += pu.loc[pu_id, 'basez']
        student_counts[fac] = count

    report['student_count'] = student_counts

    for fac, count in student_counts.items():
        print(f"  Facility {fac}: {count} students")



--- Solution #1 ---
Facilities opened: [45, 290, 317, 507, 566, 602]
Assignments:
  Facility 45 <-- Planning Units [1, 2, 19, 28, 29, 30, 55, 56, 57, 64, 65, 69, 85, 141, 144, 165, 177, 193, 196, 197, 205, 206, 210, 211, 212, 213, 214, 215, 239, 240, 344, 348, 353, 354, 369, 370, 371, 403, 404, 411, 416, 420, 429, 435, 437, 439, 441, 442, 443, 451, 453, 483, 488, 489, 501, 508, 531, 542, 549, 569, 574, 577, 581, 582, 585, 588, 592, 594, 595, 596, 597, 677, 720, 721, 736, 746, 768, 775, 797, 817, 828, 848]
  Facility 290 <-- Planning Units [6, 10, 37, 39, 47, 97, 98, 99, 100, 114, 115, 116, 118, 120, 121, 122, 125, 126, 127, 128, 132, 253, 254, 267, 269, 270, 272, 275, 276, 278, 279, 284, 285, 286, 288, 289, 290, 291, 292, 293, 294, 295, 297, 298, 307, 308, 309, 310, 382, 383, 384, 385, 392, 399, 402, 407, 421, 424, 425, 430, 434, 449, 459, 478, 479, 480, 481, 482, 485, 486, 506, 515, 526, 530, 534, 535, 536, 540, 552, 606, 607, 609, 622, 623, 630, 631, 633, 652, 654, 700, 701, 715, 71

In [80]:
# to deal with duplicate pu
for report in solution_reports:
    print(f"\n--- Solution #{report['solution_number']} ---")
    print("Facilities opened:", report['facilities'])

    print("Assignments:")
    for fac, pus in report['assignments'].items():
        print(f"  Facility {fac} <-- Planning Units {pus}")

    print("Student Count per Facility:")
    student_counts = {}
    for fac in report['facilities']:
        count = 0
        for pu_id in report['assignments'][fac]:
            basez_val = pu.loc[pu_id, 'basez']
            if isinstance(basez_val, pd.Series):
                count += basez_val.sum()
            else:
                count += basez_val
        student_counts[fac] = count

    report['student_count'] = student_counts

    for fac, count in student_counts.items():
        print(f"  Facility {fac}: {count} students")


--- Solution #1 ---
Facilities opened: [45, 290, 317, 507, 566, 602]
Assignments:
  Facility 45 <-- Planning Units [1, 2, 19, 28, 29, 30, 55, 56, 57, 64, 65, 69, 85, 141, 144, 165, 177, 191, 193, 196, 197, 205, 206, 210, 211, 212, 213, 214, 215, 239, 240, 340, 344, 348, 353, 354, 369, 370, 371, 403, 404, 411, 416, 420, 429, 435, 437, 439, 441, 442, 443, 451, 453, 483, 488, 489, 501, 508, 531, 542, 549, 569, 574, 577, 581, 582, 585, 588, 592, 594, 595, 596, 597, 677, 720, 721, 736, 746, 768, 775, 797, 817, 828, 848]
  Facility 290 <-- Planning Units [6, 10, 37, 39, 47, 97, 98, 99, 100, 114, 115, 116, 118, 120, 121, 122, 125, 126, 127, 128, 132, 253, 254, 267, 269, 270, 272, 275, 276, 278, 279, 284, 285, 286, 288, 289, 290, 291, 292, 293, 294, 295, 297, 298, 307, 308, 309, 310, 382, 383, 384, 385, 392, 399, 402, 407, 408, 421, 424, 425, 430, 434, 449, 459, 478, 479, 480, 481, 482, 485, 486, 506, 515, 526, 530, 534, 535, 536, 540, 552, 606, 607, 609, 622, 623, 630, 631, 633, 652, 654, 70

In [88]:
pu_new = pu.copy()

for solution in solution_reports: 
    facility_to_pus = solution['assignments']

    pu_to_facility = {
        pu_id: facility
        for facility, pu_list in facility_to_pus.items()
        for pu_id in pu_list
    }

    pu_new['assignment'] = pu.index.map(pu_to_facility)
    solution_number = solution['solution_number']
    pu_new.to_file(f"CFLP_hs_noSGR{solution_number}.geojson", driver="GeoJSON")

In [94]:
with open('CFLP_hs_noSGR.json', 'w') as f:
    json.dump(solution_reports, f)

In [ ]:
hs_dict = {45: 'Northern', 
          507: 'Jordan', 
          290: 'Hillside',
          567: 'Riverside',
          602: 'Southern'}

In [ ]:
# pu.to_file('CFLP_solution.geojson', driver='GeoJSON')

In [ ]:
# new_pu = gpd.read_file('/Users/leahwallihan/Durham_school_planning/DPS-Planning/GIS_files/pu_2324_SPLIT.geojson', driver='GeoJSON')